# Object Pool

In [18]:
class AnimalPool(object):
    def __init__(self, queue, auto_get=False):
        self._queue = queue
        self.item = self._queue.get() if auto_get else None
    
    def __enter__(self):
        print '__enter__'
        if self.item is None:
            self.item = self._queue.get()
        return self.item
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print '__exit__'
        if self.item is not None:
            self._queue.put(self.item)
            self.item = None
    
    def __del__(self):
        print '__del__'
        if self.item is not None:
            self._queue.put(self.item)
            self.item = None

In [19]:
try:
    import queue
except ImportError:
    import Queue as queue


animal_queue = queue.Queue()

animal_queue.put('Cat')
print 'len: ', animal_queue.qsize()
with AnimalPool(animal_queue) as animal:
    print 'Inside: ', animal

print 'len: ', animal_queue.qsize()
print 'Outside:', animal_queue.get()
print 'len: ', animal_queue.qsize()

len:  1
__enter__
Inside:  Cat
__exit__
__del__
len:  1
Outside: Cat
len:  0


In [20]:
def test_object(queue):
    pool = AnimalPool(queue, True)
    print 'Inside: ', pool.item

animal_queue.put('Dog')
test_object(animal_queue)
print 'len: ', animal_queue.qsize()
print 'Outside:', animal_queue.get()

Inside:  Dog
__del__
len:  1
Outside: Dog
